# Generate automated session reports for all participants

In [ ]:
import pandas as pd
import numpy as np
import mne
import mne_bids
import glob
import warnings

import re
import subprocess

import os
import papermill as pm
warnings.filterwarnings('ignore')

from nbconvert import HTMLExporter
from nbconvert.preprocessors import ExecutePreprocessor
from nbformat import read


### Select task parameters and file paths

In [ ]:
project_path = "/project/def-emayada/q1k/experimental/HSJ/"
pyll_path = "derivatives/pylossless/"
sync_loss_path = "derivatives/sync_loss/"
code_path = "code/q1k_sync_loss/"
out_path = ""
#out_seg_path = "epoch_fif_files/"
#out_avg_path = "erp_fif_files/"
html_reports_path = "session_reports/"
task_id = "VEP"
#subject_id = 'Q1K_HSJ_100123_F1'
run_id = '1'
session_id = '01'
site_code = 'HSJ' #'MHC' or 'HSJ'...
#sourcedata_path = "sourcedata/" 
#html_reports_path = "session_reports/" + task_id_in  + '/'



In [ ]:
# Sanity check to see which task you would like to make reports for 
print(task_id)

In [ ]:
#find existing *.edf files 
pattern = project_path + pyll_path + sync_loss_path + '**/eeg/*' + task_id + '*.edf'
processed_sessions = glob.glob(pattern, recursive=True)
        
pattern = project_path + pyll_path + '**/eeg/*' + task_id + '*.edf'
file_paths = glob.glob(pattern, recursive=True)


In [ ]:
print(f"completed files:")
processed_sessions

In [ ]:
print(f"Input files:")
file_paths

In [ ]:
# Function to extract details from filename
def extract_job_info(filename):
    # Define the pattern to capture the required sections
    pattern = r"sub-(.*?)_ses-(.*?)_task-(.*?)_run-(.*?)_eeg\.edf"
    match = re.match(pattern, filename)

    if match:
        # Extract the groups from the match
        subject_id = match.group(1)
        session_id = match.group(2)
        task_id = match.group(3)
        run_id = match.group(4)
        return subject_id, session_id, task_id, run_id
    else:
        raise ValueError("Filename pattern did not match.")


#in_pattern = project_path + pyll_path + '**/eeg/*' + task_id + '*.edf'
#in_file_paths = glob.glob(in_pattern, recursive=True)
#print(f"Input files:")
#for file_path in in_file_paths:
#    file_name = os.path.basename(file_path)  # Extract the filename from the full path
#    print(f"File Path: {file_path}, File Name: {file_name}")

    
## Make sure the output directories exists
#if not os.path.exists(project_path + pyll_path + post_path + out_seg_path + task_id_out):
#    os.makedirs(project_path + pyll_path + post_path + out_seg_path + task_id_out)
#if not os.path.exists(project_path + pyll_path + post_path + out_avg_path + task_id_out):
#    os.makedirs(project_path + pyll_path + post_path + out_avg_path + task_id_out)
    
#out_seg_pattern = project_path + pyll_path + post_path + out_seg_path + task_id_out + '/*' + task_id_out + '*epo.fif'
#out_seg_file_paths = glob.glob(out_seg_pattern, recursive=True)
#print(f"existing output epoch files:")
#for file_path in out_seg_file_paths:
#    file_name = os.path.basename(file_path)  # Extract the filename from the full path
#    print(f"File Path: {file_path}, File Name: {file_name}")

#out_avg_pattern = project_path + pyll_path + post_path + out_avg_path + task_id_out + '/*' + task_id_out + '*epo.fif'
#out_avg_file_paths = glob.glob(out_seg_pattern, recursive=True)
#print(f"existing output avg files:")
#for file_path in out_avg_file_paths:
#    file_name = os.path.basename(file_path)  # Extract the filename from the full path
#    print(f"File Path: {file_path}, File Name: {file_name}")

#    #submit_slurm_job(project_path, file_name)

## Generate html session reports for all participants

In [ ]:
#glob.glob(project_path + pyll_path + '**/*.edf')

In [ ]:
# Make sure the output directory exists
if not os.path.exists(html_reports_path):
    os.makedirs(html_reports_path)

# Create a list of sessions with errors
error_subjects = []

#Loop through existing sessions and execute the q1k_generate_individual_reports.ipynb if a session report does not already exist
#for file in glob.glob(project_path + pyll_path + "**/eeg/*" + task_id + '_*.edf'):
for file in file_paths:

    file_name = os.path.basename(file)
    print('Current data file: ' + file_name)
    ## Select anything after the Q1K and before the AEP
    ##subject_id = file.split('_')[2]
    ##site_part = file.split("HSJ_")
    subject_id = file_name.split("_")[0].split("-")[1]
    print('Participant ID: ' + subject_id)

    #subject_relation = site_part[1].split("_")[1].split("/")[0]
    #print('Participant relation: ' + subject_relation)

    #subject_id_in = subject_number + "_" + subject_relation
    #print('Participant ID input: ' + subject_id_in)
    #subject_id_out = subject_number.replace('_','').replace('-','') + subject_relation
    #print('Participant ID output: ' + subject_id_out)
    
    # Skip sessions that have already been processed
    print(subject_id)
    if subject_id in processed_sessions:
        print(subject_id + ' has already been processed')
        continue    

    # Define paths
    input_notebook = project_path + pyll_path + sync_loss_path + 'code/q1k_sync_loss/' + 'q1k_sync_loss.ipynb'

    ##output_notebook = f'./../../session_reports/{task_id_in}/executed_notebooks/{subject_id}_{task_id_in}_executed.ipynb'
    ##output_html = f'./../../session_reports/{task_id_in}/{subject_id}_{session_id}_{task_id_in}.html'
    ##output_html = f'{html_reports_dir}/{subject_id}.html'
    output_notebook = f'{project_path}{pyll_path}{sync_loss_path}{html_reports_path}executed_notebooks/{subject_id}_{task_id}_executed.ipynb'
    print('Output notebook file: ' + output_notebook)
    output_html = f'{project_path}{pyll_path}{sync_loss_path}{html_reports_path}{task_id}/{subject_id}_{task_id}.html'
    print('Output HTML file: ' + output_html)


    #print('Current data file: ' + file)
    ## Select anything after the Q1K and before the AEP
    ##subject_id = file.split('_')[2]
    #site_part = file.split(site_code + "_")
    #subject_number = site_part[1].split("_")[0]
    #print('Participant number: ' + subject_number)
    #subject_relation = site_part[1].split("_")[1].split("/")[0]
    #print('Participant relation: ' + subject_relation)

    #subject_id_in = subject_number + "_" + subject_relation
    #print('Participant ID input: ' + subject_id_in)
    #subject_id_out = subject_number.replace('_','').replace('-','') + subject_relation
    #print('Participant ID output: ' + subject_id_out)
    
    ## Skip sessions that have a session report in the output directory
    #print(subject_id_in)
    #if subject_id_in in processed_sessions:
    #    print(subject_id_in + ' has already been processed')
    #    continue    

    ## Define paths
    #input_notebook = project_path + pyll_path + postproc_path +'q1k_sync_ll_prune.ipynb'
    #print('Input notebook: ' + input_notebook)

    ## Make sure the directory exists
    #if not os.path.exists(f'{project_path}{pyll_path}{postproc_path}session_reports/{task_id}/executed_notebooks/'):
    #    os.makedirs(f'{project_path}{init_path}{html_reports_path}{task_id}/executed_notebooks/')
    
    #output_notebook = f'{project_path}{pyll_path}{postproc_path}{html_reports_path}executed_notebooks/{subject_id_in}_{task_id}_executed.ipynb'
    #print('Output notebook file: ' + output_notebook)
    #output_html = f'{project_path}{pyll_path}{postproc_path}{html_reports_path}{subject_id_in}_{task_id}.html'
    #print('Output HTML file: ' + output_html)

    try:
        # Execute the notebook
        pm.execute_notebook(input_notebook, output_notebook, kernel_name = 'q1k_env', parameters=dict(subject_id=subject_id, task_id=task_id, run_id=run_id, session_id=session_id))

        # Convert executed notebook to HTML
        html_exporter = HTMLExporter()
        html_exporter.exclude_input = True

        (body, resources) = html_exporter.from_filename(output_notebook)

        # Save HTML output
        with open(output_html, 'w', encoding='utf-8') as f:
            f.write(body)

        print(f"HTML report saved for {subject_id}.")
    

    except Exception as e:
        # Handle the error 
        error_subjects.append(subject_id)
        print(f"Error while processing {subject_id}: {e}")

# Print out the list of subjects with errors
print( "These subjects have errors: " + str(error_subjects) + " and need to be reprocessed")